In [1]:
import pandas as pd
import csv
from datetime import timedelta 

# Make clean df

In [2]:
def cleandf_csv(path_to_data):
    
# Read in

    data = pd.read_csv(path_to_data)

    cols = data.columns


# Rename columns    
    new_cols = ['timestamp', 'live', 'activity', 'main_track', 'not_main_track', 'r_activity', 'date', 'time', 'r_main_track', 'r_not_main_track', 'qual']

    # Dict for rename columns:     # old: NEW
    rename_c = {}
    for i, col in enumerate(cols):
        rename_c[cols[i]] = new_cols[i]
        
    data = data.rename(columns = rename_c)
    data.head(1)

# Replace variables

    # Safe = 0, Unsafe = 1, Not Sure = 2
    data = data.replace('EASY to practice social distancing', 0)
    data = data.replace('DIFFICULT to practice social distancing', 1)
    data = data.replace("I wasn't on the main track at Prospect Park enough to tell", 2)
    data = data.replace("I wasn't off the main track at Prospect Park enough to tell", 2)
    data = data.replace("I haven't been on the main track at Prospect Park enough to tell", 2)
    data = data.replace("I'm not off the main track at Prospect Park enough to tell", 2)

# Merge recall and non reall columns for activity, main_track, and non_main_track

    data['activity'] = data['activity'].fillna(data['r_activity'])
    data['main_track'] = data['main_track'].fillna(data['r_main_track'])
    data['not_main_track'] = data['not_main_track'].fillna(data['r_not_main_track'])
    data = data.drop(columns=['r_activity','r_main_track', 'r_not_main_track', 'timestamp'])
    
# Drop non-reported day/times:
    data = data[data['main_track'].notna()]
    data = data[data['time'].notna()]
    data = data[data['date'].notna()]

#    Create time_bin column
#    data['time'] = data['time'].apply(convert24)

    data['time_bin'] = data['date'] + " " + data['time']
    data['time_bin']= pd.to_datetime(data['time_bin']) 

    data = data.drop(columns=['date','time'])
    data['time_bin'] = BinTime(data['time_bin'])


    # Save qualitatitive answers to qual before dropping
    qual = data['qual']
    data = data.drop(columns=['qual'])
    
#     # REPLACING INDIVIDUAL ENTRIES
#     data['time_bin'][6] = data.iloc[6]['time_bin'].replace(year = 2020) 


    return data, qual

# Helper functions to clean script:

In [3]:
# HElPER FUNCTION FOR TIME CONVERSION:

# Python program to convert time 
# from 12 hour to 24 hour format 
  
# Function to convert the date format 
def convert24(str1): 
    
    if len(str1) == 10:
        str1 = "0" + str1
      
    # Checking if last two elements of time 
    # are AM and first two elements are 12 
    if str1[-2:] == "AM" and str1[:2] == "12": 
        return "00" + str1[2:-2] 
          
    # remove the AM     
    elif str1[-2:] == "AM": 
        return str1[:-2] 
      
    # Checking if last two elements of time 
    # is PM and first two elements are 12    
    elif str1[-2:] == "PM" and str1[:2] == "12": 
        return str1[:-2] 
      
    # For when respondents entered 24 hour time format
    # for a PM time (we don't add 12)
    elif (int(str1[:2]) + 12) > 12:
        return str1[:-2]
    
    else:
    # add 12 to hours and remove PM 
        return str(int(str1[:2]) + 12) + str1[2:8] 
  
# Driver Code         
print(convert24("08:05:45 PM")) 

08:05:45 


In [4]:
## HUAYI's script for binning: 
# https://github.com/four-insightfellows-corona-project/PopularTimes/blob/master/Scp/BinTime.py


def binMinute(minute):
    '''
    Calculate which of the 15min bins the current minute falls in.
    The bins are 05, 20, 35, 50. 
    Current minutes less than 05 belong to bin 50 of the hour before. 
    '''    
    m2q= (minute-5)/15
    if m2q < 0:
        newminute = -10 
    elif m2q < 1:
        newminute = 5
    elif m2q < 2:
        newminute = 20
    elif m2q < 3:
        newminute = 35
    elif m2q < 4:
        newminute = 50
    print("m2q", m2q, "minute", minute)
    return newminute       
    
def BinTime(df_datetime_col):
    '''
    Take a df column that contains datetime object (df_datetime_col)
    Return a new column that contains the binned datetime object
    The bins are as defined in binMinute
    '''
    binned_col= df_datetime_col.apply(lambda x: x.replace(minute = 0) + timedelta(minutes=binMinute(x.minute)))
    return binned_col

## Check output and write csv

In [5]:
# CHECK:

path_to_data = "../model/alldata/survey_raw.csv"
data, qual = cleandf_csv(path_to_data)


# WRITE

data.to_csv('survey_clean.csv', index = False, header=True)


print(data.shape)


# CHECK OUTPUT and write csv
# print(len(data.columns))
# cols = data.columns
# print(cols)
new_cols = ['timestamp', 'live', 'activity', 'main_track', 'not_main_track', 'r_activity', 'date', 'time', 'r_main_track', 'r_not_main_track', 'qual']




m2q 1.6666666666666667 minute 30
m2q -0.3333333333333333 minute 0
m2q 1.6666666666666667 minute 30
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q 3.0 minute 50
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q 1.6666666666666667 minute 30
m2q 1.6666666666666667 minute 30
m2q -0.3333333333333333 minute 0
m2q 1.6666666666666667 minute 30
m2q -0.3333333333333333 minute 0
m2q 1.6666666666666667 minute 30
m2q -0.3333333333333333 minute 0
m2q 1.6666666666666667 minute 30
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q 1.6666666666666667 minute 30
m2q -0.3333333333333333 minute 0
m2q 2.6666666666666665 minute 45
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q 0.6666666666666666 minute 15
m2q 2.6666666666666665 minute 45
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 m

In [6]:
path_to_data = "../model/alldata/survey_raw.csv"
data, qual = cleandf_csv(path_to_data)
data.shape

m2q 1.6666666666666667 minute 30
m2q -0.3333333333333333 minute 0
m2q 1.6666666666666667 minute 30
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q 3.0 minute 50
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q 1.6666666666666667 minute 30
m2q 1.6666666666666667 minute 30
m2q -0.3333333333333333 minute 0
m2q 1.6666666666666667 minute 30
m2q -0.3333333333333333 minute 0
m2q 1.6666666666666667 minute 30
m2q -0.3333333333333333 minute 0
m2q 1.6666666666666667 minute 30
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q 1.6666666666666667 minute 30
m2q -0.3333333333333333 minute 0
m2q 2.6666666666666665 minute 45
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q 0.6666666666666666 minute 15
m2q 2.6666666666666665 minute 45
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 minute 0
m2q -0.3333333333333333 m

(135, 5)